In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb

In [5]:
df_train = pd.read_csv('train_ready.csv', index_col = 'building_id')
df_test = pd.read_csv('test_ready.csv', index_col = 'building_id')

In [6]:
#split
X = df_train.drop(columns=['damage_grade'])
y = df_train['damage_grade']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3 , random_state=101)

In [10]:
# classificationmodel
classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.05,max_depth=20, seed=101)

In [12]:
calibrate = CalibratedClassifierCV(classifier, cv=5)

In [13]:
calibrate.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=XGBClassifier(base_score=0.5,
                                                    booster='gbtree',
                                                    colsample_bylevel=1,
                                                    colsample_bynode=1,
                                                    colsample_bytree=1, gamma=0,
                                                    learning_rate=0.05,
                                                    max_delta_step=0,
                                                    max_depth=20,
                                                    min_child_weight=1,
                                                    missing=None,
                                                    n_estimators=100, n_jobs=1,
                                                    nthread=None, num_class=3,
                                                    objective='multi:softmax',
                                                    

In [16]:
y_pred = calibrate.predict(X_test)
y_pred_train = calibrate.predict(X_train)

In [17]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           1       0.98      0.85      0.91     17488
           2       0.90      0.96      0.93    103652
           3       0.93      0.85      0.89     61280

    accuracy                           0.91    182420
   macro avg       0.94      0.89      0.91    182420
weighted avg       0.92      0.91      0.91    182420



In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.70      0.45      0.55      7636
           2       0.74      0.85      0.79     44607
           3       0.76      0.63      0.68     25938

    accuracy                           0.74     78181
   macro avg       0.73      0.65      0.68     78181
weighted avg       0.74      0.74      0.73     78181



In [19]:
predict = calibrate.predict(df_test)

In [21]:
submit = pd.read_csv('submission_format.csv', index_col='building_id')

In [22]:
submit.head()

,damage_grade
building_id,
300051,1
99355,1
890251,1
745817,1
421793,1


In [23]:
submit.drop(columns=['damage_grade'], inplace=True)

In [24]:
submit['damage_grade'] = predict

In [25]:
submit.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,2
421793,3


In [26]:
submit.to_csv('final.csv')

### CalibratedClassifierCV improved model performance
Micro average F1 score incread by 1%